   Input files

In [ ]:
obj_file = f"C1_volume.obj" # file containing 3D cell mesh
file_1 = f"C1_1a_xyz.txt" # file containing RNA spots for condition 1
file_2 = f"C1_1b_xyz.txt" # file containing RNA spots for condition 2
coloc_threshold = 0.5 # threshold for colocalization: center to center distance (in microns)
# Set working directory
import os
os.chdir("/Users/sudharsankannan/Library/CloudStorage/OneDrive-UW-Madison/Taylor/data")    # Set your working directory which contains the data files

Load required libraries or install if missing

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import open3d as o3d
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
import gc
from multiprocessing import Pool, cpu_count
import igl
from scipy.spatial.distance import cdist
from scipy.stats import gaussian_kde
import trimesh

Functions for randomization and coloclaization analysis

In [3]:

# Function to extract RNA spots from text file
def extract_rna_spots(file):
    rna_data = pd.read_csv(file, sep='\t')
    print(rna_data.columns)  # Print the column names to inspect them
    #rna_spots = rna_data[['Position X', 'Position Y', 'Position Z']].values
    rna_spots = rna_data.iloc[:, :3].values
    rna_spots = np.round(rna_spots, 4)
    return rna_spots


# Function to filter spots inside the mesh
def filter_spots_inside_mesh(mesh, spots): ##update this using winding number
    spots = spots[np.all(np.isfinite(spots), axis=1)]
    # Get the bounding box of the mesh
    min_bound, max_bound = mesh.bounds
    
    # Filter spots to be within the bounding box
    within_bounds = np.all((spots >= min_bound) & (spots <= max_bound), axis=1)
    spots_within_bounds = spots[within_bounds]
    
    # Check if the remaining spots are inside the mesh
    inside = mesh.contains(spots_within_bounds)
    
    return spots_within_bounds[inside]

def colocper(spots1, spots2, threshold=0.5):
    # Calculate pairwise Euclidean distances
    distances = cdist(spots1, spots2)
    # Count the number of spots in spots1 within the threshold distance of spots2
    close_spots1 = np.any(distances <= threshold, axis=1)
    percentage_spots1 = np.mean(close_spots1) * 100
    # Count the number of spots in spots2 within the threshold distance of spots1
    close_spots2 = np.any(distances <= threshold, axis=0)
    percentage_spots2 = np.mean(close_spots2) * 100
    result_dict = {
        'coloc_1_with_2': percentage_spots1,
        'coloc_2_with_1': percentage_spots2
    }
    return result_dict



def generate_random_spots(mesh, desired_points):
    # Get the bounding box
    min_bound, max_bound = mesh.bounds

    points_inside_mesh = []

    while len(points_inside_mesh) < desired_points:
        # Sample more points
        num_points_to_generate = (desired_points - len(points_inside_mesh))
        random_points = np.random.uniform(low=min_bound, high=max_bound, size=(num_points_to_generate, 3))
        inside = mesh.contains(random_points)
        points_inside_mesh.extend(random_points[inside])
    #convert to np array
    points_inside_mesh = np.array(points_inside_mesh)
    
    return points_inside_mesh


def calculate_max_density(rna_spots, mesh, resolution=1.0):
    # Get the bounding box
    min_bound, max_bound = mesh.bounds

    # Calculate the number of bins along each axis
    num_bins = np.ceil((max_bound - min_bound) / resolution).astype(int)

    # Initialize a 3D grid to count the number of spots in each cubic micrometer
    density_grid = np.zeros(num_bins, dtype=int)

    # Shift the spots so that the bounding box starts at the origin
    shifted_spots = rna_spots - min_bound

    # Calculate the bin indices for each spot
    bin_indices = np.floor(shifted_spots / resolution).astype(int)

    # Count the number of spots in each bin
    for idx in bin_indices:
        if np.all(idx >= 0) and np.all(idx < num_bins):
            density_grid[tuple(idx)] += 1

    # Calculate the density for each cubic micrometer
    density_grid = density_grid / (resolution ** 3)

    # Find the maximum density
    max_density = np.max(density_grid)

    return max_density

def calculate_max_density_bbox(rna_spots, min_bound, max_bound, resolution=1.0):
    """
    Calculates the maximum density (spots/µm³) within a 3D bounding box grid.

    Parameters:
        rna_spots (np.ndarray): Nx3 array of RNA spot coordinates.
        min_bound (np.ndarray): 3-element array of min x, y, z values of bounding box.
        max_bound (np.ndarray): 3-element array of max x, y, z values of bounding box.
        resolution (float): Grid resolution in microns.

    Returns:
        float: Maximum local density in spots per µm³.
    """
    # Calculate the number of bins along each axis
    num_bins = np.ceil((max_bound - min_bound) / resolution).astype(int)

    # Initialize a 3D grid
    density_grid = np.zeros(num_bins, dtype=int)

    # Shift the RNA spots to align with the origin of the bounding box
    shifted_spots = rna_spots - min_bound

    # Determine which bin each spot falls into
    bin_indices = np.floor(shifted_spots / resolution).astype(int)

    # Accumulate spot counts
    for idx in bin_indices:
        if np.all(idx >= 0) and np.all(idx < num_bins):
            density_grid[tuple(idx)] += 1

    # Convert counts to densities
    density_grid = density_grid / (resolution ** 3)

    # Return the maximum density
    return np.max(density_grid)

def generate_kde_spots(rna_spots, mesh):
    # Estimate KDE for rna_spots
    kde_model = gaussian_kde(rna_spots.T)
   
    # Get the bounding box of the mesh
    min_bound, max_bound = mesh.bounds
    bbox_volume = np.prod(max_bound - min_bound)

    spots_per_cubi_um = calculate_max_density(rna_spots, mesh)

    desired_points = int(bbox_volume * spots_per_cubi_um/2)
    sample_space = np.random.uniform(low=mesh.bounds[0], high=mesh.bounds[1], size=(desired_points, 3))

    V = np.array(mesh.vertices)
    F = np.array(mesh.faces)
    # Compute winding number for the sample space
    W = igl.fast_winding_number_for_meshes(V, F, sample_space)
    # Keep only the points where winding number >= 0.5 (i.e., inside the mesh)
    inside_mask = W >= 0.5
    points_inside_mesh1 = sample_space[inside_mask]

    #change to np array
    points_inside_mesh = np.array(points_inside_mesh1)

    # Evaluate KDE on the sampled points
    kde_eval = kde_model(points_inside_mesh.T)

    # Normalize KDE values to create probability weights
    probabilities = kde_eval / np.sum(kde_eval)

    # Sample points based on KDE probability distribution
    sampled_indices = np.random.choice(len(points_inside_mesh), len(rna_spots), replace=False, p=probabilities)
    KDE_spots = points_inside_mesh[sampled_indices]

    return KDE_spots
    


Code for randomization and colocalization analysis

In [4]:
rna_spots_1 = extract_rna_spots(file_1)
rna_spots_2 = extract_rna_spots(file_2)
mesh = trimesh.load_mesh(obj_file)

# Original colocalization
colocog = colocper(rna_spots_1, rna_spots_2, threshold=coloc_threshold)

# Uniform random
random_rna_spots_1 = generate_random_spots(mesh, desired_points=len(rna_spots_1))
random_rna_spots_2 = generate_random_spots(mesh, desired_points=len(rna_spots_2))
random_result = colocper(random_rna_spots_1, random_rna_spots_2, threshold=coloc_threshold)

# KDE-based sampling
kde_randomized_rna_spots_1 = generate_kde_spots(rna_spots_1, mesh)
kde_randomized_rna_spots_2 = generate_kde_spots(rna_spots_2, mesh)
kde_result = colocper(kde_randomized_rna_spots_1, kde_randomized_rna_spots_2, threshold=coloc_threshold)


gc.collect()

# === Combine results into a small table ===
coloc_1_with_2_table = pd.DataFrame([{
    "colocog": colocog["coloc_1_with_2"],
    "randomly_random": random_result["coloc_1_with_2"],
    "kde_sampling": kde_result["coloc_1_with_2"],
}])

coloc_2_with_1_table = pd.DataFrame([{
    "colocog": colocog["coloc_2_with_1"],
    "randomly_random": random_result["coloc_2_with_1"],
    "kde_sampling": kde_result["coloc_2_with_1"],
}])

# === Output ===
print("\nTable for coloc_1_with_2:\n", coloc_1_with_2_table)
print("\nTable for coloc_2_with_1:\n", coloc_2_with_1_table)

Index(['Position.X', 'Position.Y', 'Position.Z'], dtype='object')
Index(['Position.X', 'Position.Y', 'Position.Z'], dtype='object')

Table for coloc_1_with_2:
    colocog  randomly_random  kde_sampling
0      0.0              0.0           0.0

Table for coloc_2_with_1:
    colocog  randomly_random  kde_sampling
0      0.0              0.0           0.0
